# Developing regression tests

Comparing output of old (2014, `src/cmm_error_map/mpl_2014`) matplotlib code with new code.

Will write these up as pytest tests.

In [16]:
import numpy as np

import cmm_error_map.config.config as cf
import cmm_error_map.data_cmpts as dc
import cmm_error_map.mpl_2014.design_matrix_linear as design_old

In [17]:
model_params = dc.model_parameters_dict.copy()
"""
regression test the  a set of parameter with a Koba plate on the XY plane
against the old matplotlib code
"""
# XY plane
x0, y0, z0 = 250.0, 50.0, 50.0
x0xy, y0xy, z0xy = x0, y0, z0
transform_mat_xy = np.array(
    [
        [1.0, 0.0, 0.0, x0xy],
        [0.0, 1.0, 0.0, y0xy],
        [0.0, 0.0, 1.0, z0xy],
        [0.0, 0.0, 0.0, 1.0],
    ]
)

xt, yt, zt = 0.0, 0.0, -243.4852

# new code
cmm = dc.pmm_866
prb_xy = dc.Probe(title="P0", name="p0", length=np.array([xt, yt, zt]))

mmt = dc.Measurement(
    title="Plate XY",
    name="mmt_00",
    artefact=cf.artefact_models["KOBA 0620"],
    transform_mat=transform_mat_xy,
    probe=prb_xy,
    cmm_nominal=None,
    cmm_dev=None,
    mmt_nominal=None,
    mmt_dev=None,
)

mmt.recalculate(model_params, cmm.cmm_model)

# matplotlib code
params = list(model_params.values())
dxy, eXYZ, XYZp, XM, plate_nom = design_old.modelled_mmts_XYZ(
    transform_mat_xy, xt, yt, zt, params, verbose=True
)


need to check `mmt.cmm_nominal`, `mmt.cmm_dev`, `mmt.mmt_nominal` and `mmt.mmt_dev` against old code

In [18]:
# mmt.cmm_nominal is nominals in machine csy - new code
# XM is nominals in machine csy - old code
print(f"{mmt.cmm_nominal.shape=} \n{XM[:-1, :].shape=}")

mmt.cmm_nominal.shape=(3, 25) 
XM[:-1, :].shape=(3, 25)


In [19]:
np.testing.assert_allclose(XM[:-1, :], mmt.cmm_nominal)

In [20]:
# mmt.cmm_dev is deformed minus nominal position in CMM CSY
# eXYZ is error in CMM CSY
print(f"{mmt.cmm_dev.shape=} \n{eXYZ[:-1,:].shape=}")

mmt.cmm_dev.shape=(3, 25) 
eXYZ[:-1,:].shape=(3, 25)


In [21]:
np.testing.assert_allclose(eXYZ[:-1, :], mmt.cmm_dev)

In [22]:
# mmt.mmt_nominal is the nominal position of the balls in artefact CSY
# plate_nom
print(f"{mmt.mmt_nominal[:-1, :].shape=} \n{plate_nom.shape=}")

mmt.mmt_nominal[:-1, :].shape=(2, 25) 
plate_nom.shape=(2, 25)


In [23]:
np.testing.assert_allclose(plate_nom, mmt.mmt_nominal[:-1, :])

In [24]:
# mmt.mmt_dev is the deformed minus nominal position in artefact CSY
# dxy
print(f"{mmt.mmt_dev[:-1,:].shape=} \n{dxy.T.shape=}")

mmt.mmt_dev[:-1,:].shape=(2, 25) 
dxy.T.shape=(2, 25)


In [25]:
np.testing.assert_allclose(dxy.T, mmt.mmt_dev[:-1, :], atol=1e-6)

In [26]:
# as  a function
def test_XY(model_params: dict[str, float], verbose=False, signs={"old": 1, "new": 1}):
    """
    regression test the  a set of parameter with a Koba plate on the XY plane
    against the old matplotlib code
    """
    # XY plane
    x0, y0, z0 = 250.0, 50.0, 50.0
    x0xy, y0xy, z0xy = x0, y0, z0
    transform_mat_xy = np.array(
        [
            [1.0, 0.0, 0.0, x0xy],
            [0.0, 1.0, 0.0, y0xy],
            [0.0, 0.0, 1.0, z0xy],
            [0.0, 0.0, 0.0, 1.0],
        ]
    )

    xt, yt, zt = 0.0, 0.0, -243.4852

    # new code
    cmm = dc.pmm_866
    prb_xy = dc.Probe(title="P0", name="p0", length=np.array([xt, yt, zt]))

    mmt = dc.Measurement(
        title="Plate XY",
        name="mmt_00",
        artefact=cf.artefact_models["KOBA 0620"],
        transform_mat=transform_mat_xy,
        probe=prb_xy,
        cmm_nominal=None,
        cmm_dev=None,
        mmt_nominal=None,
        mmt_dev=None,
    )
    signed_mp = {key: signs["new"] * value for key, value in model_params.items()}
    mmt.recalculate(signed_mp, cmm.cmm_model)

    # matplotlib code
    params = list(model_params.values())
    signed_params = [value * signs["old"] for value in params]
    dxy, eXYZ, XYZp, XM, plate_nom = design_old.modelled_mmts_XYZ(
        transform_mat_xy, xt, yt, zt, signed_params, verbose=True
    )
    if verbose:
        print(signed_mp)
        print(signed_params)
        print(
            f"cmm dev difference\n max {(eXYZ[:-1, :] - mmt.cmm_dev).max()}, min {(eXYZ[:-1, :] - mmt.cmm_dev).min()}"
        )
        print(
            f"mmt dev difference\n max {(dxy.T - mmt.mmt_dev[:-1, :]).max()}, min {(dxy.T - mmt.mmt_dev[:-1, :]).min()}"
        )

        print(f"{eXYZ[:-1, :]=}")
        print(f"{mmt.cmm_dev=}")

        print(f"{dxy.T=}")
        print(f"{mmt.mmt_dev=}")

    np.testing.assert_allclose(XM[:-1, :], mmt.cmm_nominal, atol=1e-6)
    np.testing.assert_allclose(eXYZ[:-1, :], mmt.cmm_dev, atol=1e-6)
    np.testing.assert_allclose(plate_nom, mmt.mmt_nominal[:-1, :], atol=1e-6)
    np.testing.assert_allclose(dxy.T, mmt.mmt_dev[:-1, :], atol=1e-6)
    return eXYZ[:-1, :], mmt.cmm_dev, dxy.T, mmt.mmt_dev[:-1, :]

In [27]:
# the above null case
model_params = dc.model_parameters_dict.copy()
old_cmm_dev, new_cmm_dev, old_mmt_dev, new_mmt_dev = test_XY(model_params)


In [28]:
# Txx
model_params = dc.model_parameters_dict.copy()
model_params["Txx"] = 1e-6
old_cmm_dev, new_cmm_dev, old_mmt_dev, new_mmt_dev = test_XY(model_params)

In [29]:
# Tyy
model_params = dc.model_parameters_dict.copy()
model_params["Tyy"] = 1e-6
old_cmm_dev, new_cmm_dev, old_mmt_dev, new_mmt_dev = test_XY(model_params)

In [30]:
# Tzz
model_params = dc.model_parameters_dict.copy()
model_params["Tzz"] = 1e-6
old_cmm_dev, new_cmm_dev, old_mmt_dev, new_mmt_dev = test_XY(model_params)

In [31]:
# Txy
model_params = dc.model_parameters_dict.copy()
model_params["Txy"] = 1e-6
old_cmm_dev, new_cmm_dev, old_mmt_dev, new_mmt_dev = test_XY(model_params, verbose=True)

{'Txx': 0.0, 'Txy': 1e-06, 'Txz': 0.0, 'Tyx': 0.0, 'Tyy': 0.0, 'Tyz': 0.0, 'Tzx': 0.0, 'Tzy': 0.0, 'Tzz': 0.0, 'Rxx': 0.0, 'Rxy': 0.0, 'Rxz': 0.0, 'Ryx': 0.0, 'Ryy': 0.0, 'Ryz': 0.0, 'Rzx': 0.0, 'Rzy': 0.0, 'Rzz': 0.0, 'Wxy': 0.0, 'Wxz': 0.0, 'Wyz': 0.0}
[0.0, 1e-06, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
cmm dev difference
 max 0.0, min -0.0015640000000151594
mmt dev difference
 max 2.2737367544323206e-13, min -0.0010640000000137206
eXYZ[:-1, :]=array([[ 0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
         0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
         0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
         0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
         0.      ],
       [-0.00025 , -0.000383, -0.000516, -0.000649, -0.000782, -0.00025 ,
        -0.000383, -0.000516, -0.000649, -0.000782, -0.00025 , -0.000383,
        -0.000516, -0.000649,

AssertionError: 
Not equal to tolerance rtol=1e-07, atol=1e-06

Mismatched elements: 25 / 75 (33.3%)
Max absolute difference: 0.001564
Max relative difference: 2.
 x: array([[ 0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
         0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,
         0.      ,  0.      ,  0.      ,  0.      ,  0.      ,  0.      ,...
 y: array([[0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
        0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,
        0.      , 0.      , 0.      , 0.      , 0.      , 0.      ,...

In [33]:
# mess with signs Txy
model_params = dc.model_parameters_dict.copy()
model_params["Txy"] = 1e-6
old_cmm_dev, new_cmm_dev, old_mmt_dev, new_mmt_dev = test_XY(
    model_params, signs={"old": 1, "new": -1}
)